In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.getLogger("httpx").setLevel(logging.WARNING)

import sys
sys.path.append('../../')
from src.df_llm_processor import run_pipeline

import pandas as pd
from tqdm import tqdm
tqdm.pandas()


job_id='dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc'

df = pd.read_csv(f"../../extractions/{job_id}/co_articles.csv")
df

,source_name,title,company_codes,date,region_codes,article_fragment,num_segments,an,tile_id,company_names,is_carve_out,target_company_code,subsidiary_company_code,is_relevant,carve_out_stage,reasoning
0,Investors Chronicle - Magazine and Web Content,Why the end of De La Rue is the end of an era,"cryamb,dlar,pskdzt,schr",2025-01-17 07:25:09.485,"eurz,uk,weurz","\n\nMeanwhile, Crystal Amber’s shareholders ha...",6,INVR000020250116el1g0002t,1912118,"Crystal Amber Fund Limited,De La Rue PLC,Crane...",True,dlar,pskdzt,True,Late,De La Rue is currently engaged in selling off ...
1,Reuters News,UPDATE 1-Getir founder vows to sue over Mubada...,mbdlic,2025-01-16 07:19:46.849,"asiaz,balkz,devgcoz,dvpcoz,eurz,gulfstz,meastz...","\n\nIn September, Turkey's competition board s...",5,LBA0000020250115el1f01blx,1912168,Mubadala Investment Company PJSC,True,mbdlic,getir,True,Early,Mubadala is acquiring Getir's profitable local...
2,Western Morning News,Sutton Harbour Group given new deadlines to pa...,"dfvndk,nwb,rbsct,suttoh",2025-01-24 07:22:24.769,"eland,eurz,uk,weurz","The company, which in December 2024 announced ...",3,WMN0000020250123el1n00012,1913466,"Sky Harbour Group Corp.,National Westminster B...",True,suttoh,NaN,True,Early,Sutton Harbour Group PLC is undertaking a debt...
3,CE NoticiasFinancieras,"""My goal is not for Castlelake to sell, but to...","binter,foursh,gosic,tpgcml",2025-01-23 07:25:51.277,"africaz,balea,canry,eecz,eurz,medz,nafrz,spain...","\n\nQ: But it is for sale.\n\nA: Well, I don't...",31,NFINCE0020250122el1m003k1,1913645,"Bankinter SA,Four Seasons Hotels Limited,GIC P...",True,binter,NaN,True,Early,The company (Bankinter SA) is discussing asset...
4,CE NoticiasFinancieras,"""My goal is not for Castlelake to sell, but to...","binter,foursh,gosic,tpgcml",2025-01-23 07:25:51.277,"africaz,balea,canry,eecz,eurz,medz,nafrz,spain...","\n\nBorja Escalada (Madrid, 1979) took the rei...",31,NFINCE0020250122el1m003k1,1913663,"Bankinter SA,Four Seasons Hotels Limited,GIC P...",True,gosic,NaN,True,Early,The new CEO of Millenium Hospitality (gosic) h...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2864,GlobalData Company Profiles,Comcast Corp - History,"akmt,amce,amdcs,amronl,amulci,amzcom,applc,apu...",2025-05-16 07:24:23.764,"atlnt,eurz,namz,uk,usa,usga,uss,weurz",\n\nDate : 2023\n\nNew Products/Services : In ...,62,GLOMCP0020250515el5f00etv,2936780,"Akamai Technologies, Inc.,AMC Entertainment Ho...",True,comcst,hulul,True,Early,Comcast Corporation (comcst) announced plans t...
2865,Public Companies News and Documents via PUBT,Canal+ SA - 2024 Annual Report (CANAL Annual R...,"amronl,dmot,dsnyw,fratv,gneau,hisgcl,lgelec,ls...",2025-05-08 07:35:21.098,"devgcoz,eecz,eeurz,eurz,fra,medz,namz,pol,uk,u...",\n\nperformers and are already contributing po...,45,LCDVP00020250507el5701ee2,2936872,"Warner Bros. Discovery, Inc.,Dailymotion SA,Th...",True,gneau,NaN,True,Early,Vivendi SA is implementing significant cost-cu...
2866,The Irish Times,PTSB chief praises takeover target Finance Ire...,"dsbnk,ncnbco,wesah",2025-05-11 07:24:43.401,"eecz,eurz,ire,weurz",\n\nFinance Ireland reported a day earlier tha...,6,IRTI000020250510el5a0001k,2937352,"permanent tsb,Bank of America Corporation,RBC ...",True,dsbnk,ncnbco,True,Early,"Finance Ireland, majority-owned by Pimco and p..."
2867,Sharecast,Synthomer offloads inorganic chemistry busines...,"heqned,yulc",2025-05-07 07:30:32.306,"eurz,uk,weurz",The London-listed firm noted that William Blyt...,2,SHAREC0020250506el560008d,2937621,"H2 Equity Partners,Synthomer PLC",True,yulc,heqned,True,Early,"Synthomer PLC designated William Blythe, part ..."


## Providing summary with more details and NC sectors mapping

In [3]:
from typing import Literal
from pydantic import BaseModel,Field
from llm_utils.factory import LLMChainFactory
from langchain_core.prompts import PromptTemplate

business_request = """
### Deals search criteria
* Completed date (last 10 years)
* Geography (Europe) - divestment opportunities have to be in the EEA

* Deal technique (Divestment)
* Sector (Financial services)
* Size (TBD)

Note that we are looking for future carve-outs, so please do not include any deals that have already been completed.
"""

class CarveOutIdentificationSummary(BaseModel):
    target_company: str = Field(description="Name of the company that may consider divesting a subsidiary")
    group: str = Field(description="Ultimate parent financial group of the target company")
    group_hq: str = Field(description="Headquarters of the ultimate parent financial group; two-letter country code")
    vertical: str = Field(description="Sector of the financial group, i.e. Banking, Insurance, Data, etc.")
    
    potential_disposal: str = Field(description="Potential subdivision to be disposed of, i.e. UK business, Insurance Arm, IP business, etc - specific parts of the business")
    potential_disposal_company: str = Field(description="Name of the specific subsidiary or business unit considered for disposal")
    potential_disposal_country: str = Field(description="EEA country where the potential disposal company is based; two-letter country code")  
    disposal_nc_sector: Literal["Financial Services", "Technology & Payments", "Healthcare","Services & Industrial Tech", "Other"] = Field(description="The specific NC sector applicable to the disposal company")

    article_quote: str = Field(description="A relevant direct quote from the article supporting the carve-out rationale")    
    relevant: bool = Field(description="Boolean indicator if the article meets regional criteria for potential disposals within the EEA")
    interest_score: float = Field(description="Interest level of carve-out opportunity between 0 (low) to 1 (high), based on strategic timing and early-stage indicators")
    rationale: str = Field(description="Brief rationale (1-2 sentences) for why the carve-out opportunity exists (e.g., strategic refocusing, divestment of non-core assets)")

identification_prompt = """
You are an expert investment professional identifying corporate carve-out opportunities in the financial services sector on behalf of Nordic Capital (NC). NC operates only in the EEA and the US—disregard other regions for disposals.

Given the provided news article, perform the following:
1. **Identify** clearly:
    - **Target Company:** The company potentially divesting a subsidiary.
    - **Financial Group:** Ultimate parent group of the target company.
    - **Financial Group HQ:** Two-letter country code (e.g., UK, DE).
    - **Potential Disposal Company:** Subsidiary/unit explicitly or implicitly mentioned as potentially disposable.
    - **Potential Disposal Country:** Two-letter EEA country code of the disposal company.
    - **Disposal NC Sector:** Select exactly from "Financial Services", "Technology & Payments", "Healthcare", "Services & Industrial Tech", "Other".

2. **Assess** clearly:
    - **Relevant:** True if the potential disposal is within the EEA region, else False.
    - **Interest Score:** Rate interest from 0.0 (low) to 1.0 (high), higher if signals are early-stage, strategic reviews, or management changes.
    
3. **Provide** succinct reasoning:
    - **Rationale:** 1–2 sentences explicitly summarizing why the carve-out may occur (e.g., divestment of non-core assets, strategic refocusing).
    - **Article Quote:** Provide a direct, supportive quote from the article.

### Carve-out Identification Guidelines:

- **Identify as carve-out if at least ONE:**
    - Explicit intention to divest subsidiaries.
    - Strategic refocusing implying disposals.
    - Management evaluating options for specific business segments.
    - Simplifying corporate structure.
    - Divesting non-core or underperforming units.
    
- **DO NOT identify as carve-out:**
    - Simple stake sales or IPO plans without explicit business unit separation.
    - Already completed transactions (note in rationale if relevant, but mark lower interest).
    - General M&A or opinion articles without concrete signals.
    
- **Prioritize future carve-out opportunities.**
- **Dismiss articles with past transactions. Now is 2025, news with deals from 2023 are outdated.**

Business request: {business_request}

### Response Format:
If information is unclear or missing, explicitly state "Information Not Available" for that field.

Article and current assessment:

source: {source_name}
title: {title}
date: {date}
article body: {article_fragment}
mentioned_companies: {companies}
mentioned_company_codes: {company_codes}
target_company_code: {target_company_code}
subsidiary_company_code: {subsidiary_company_code}
carve_out_stage: {carve_out_stage}
carve_out_reasoning: {reasoning} 
"""


factory = LLMChainFactory(model_name='o4-mini', provider="openai")
summary_template = PromptTemplate.from_template(template=identification_prompt, partial_variables={'business_request': business_request})
runnable = factory.build_search_runnable_with_structured_output(pydantic_model=CarveOutIdentificationSummary)
summary_chain = summary_template | runnable

In [4]:
summary_cols_mapping = {
    "company_names": "companies",
}
summary_df_results = await run_pipeline(
    df[:],
    summary_chain,
    partial_dir=f"../../extractions/{job_id}/summarization/partial_outputs",
    final_path=f"../../extractions/{job_id}/summarization/summary_results",
    max_retries=4,
    initial_delay=3.0,
    partial_every=2800,
    cols_mapping=summary_cols_mapping,
    cols2append_mapping={'date':'date'}
)

Rows processed:  98%|█████████▊| 2801/2869 [08:08<00:10,  6.48row/s]

[checkpoint] 2800 rows → ../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc/summarization/partial_outputs_2025-05-22_14-39-38


Rows processed: 100%|██████████| 2869/2869 [13:18<00:00,  3.59row/s]
INFO:src.df_llm_processor.processor:Processing complete: 2869 rows processed
INFO:src.df_llm_processor.processor:[done] 2869 rows saved → ../../extractions/dj-synhub-extraction-lkbi9fy6zepu8rcjuxqhjwkbld52wgt0-ouutovygqc/summarization/summary_results_2025-05-22_14-39-38.csv


In [5]:
summary_df = summary_df_results.copy() #to torture it
summary_df = summary_df[summary_df.relevant]
summary_df = summary_df.set_index('index').sort_index()
summary_df

,target_company,group,group_hq,vertical,potential_disposal,potential_disposal_company,potential_disposal_country,disposal_nc_sector,article_quote,relevant,interest_score,rationale,date
index,,,,,,,,,,,,,
0,De La Rue PLC,De La Rue Group,UK,Financial Services,Remaining Group Assets,Remaining Business Units,UK,Financial Services,De La Rue announced a conditional sale of its ...,True,0.8,De La Rue is divesting parts of its business t...,2025-01-17 07:25:09.485
5,Uniper SE,Federal Government of Germany,DE,Energy,Government Stake,Uniper SE,DE,Other,The German government has engaged advisors as ...,True,0.8,The German government is evaluating options to...,2025-01-16 07:24:38.411
8,Brait,Brait SE,LU,Financial Services,Fitness Chain,Virgin Active,GB,Financial Services,Will Brait unbundle its holding in consumer br...,True,0.9,Brait is considering unbundling its holding in...,2025-01-25 07:21:18.807
9,Anglo American PLC,Anglo American,UK,Mining,Unbundling and separate listing,De Beers,GB,Other,An unbundling and separate (re)listing of this...,True,0.9,Anglo American is considering unbundling De Be...,2025-01-25 07:21:18.807
17,Hummingbird Resources PLC,CIG Parties,UK,Financial Services,Non-core or duplicative operations,Information Not Available,UK,Financial Services,The CIG Parties intend to simplify the Humming...,True,0.8,The CIG Parties plan to simplify Hummingbird's...,2025-01-14 07:07:58.244
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2859,Banco Santander SA,Banco Santander SA,ES,Banking,Polish Operations,Santander Bank Polska SA,PL,Financial Services,Banco Santander SA on Monday confirmed that it...,True,0.6,Banco Santander SA is considering selling a si...,2025-05-08 07:33:18.578
2860,Santander Bank Polska S.A.,Banco Santander S.A.,ES,Banking,Polish subsidiary,Santander Bank Polska S.A.,PL,Financial Services,Santander announces the sale of 49% of Santand...,True,0.8,Santander is divesting a significant stake in ...,2025-05-08 07:33:18.578
2865,Canal+ SA,Vivendi SA,FR,Media,Information Not Available,Information Not Available,FR,Other,Vivendi SA is implementing significant cost-cu...,True,0.7,Vivendi SA is focusing on cost-cutting and ope...,2025-05-08 07:35:21.098


In [6]:
summary_df.target_company.value_counts()

target_company
Galapagos NV                                              40
UBS Group AG                                              29
Banco Santander SA                                        25
De La Rue PLC                                             24
NatWest Group PLC                                         23
                                                          ..
Evoke PLC                                                  1
Scopevisio                                                 1
Exolum (Compania Logistica de Hidrocarburos CLH, S.A.)     1
Baloise Holding AG                                         1
Bet365 Group Limited                                       1
Name: count, Length: 848, dtype: int64

In [7]:
full_df = df[['source_name','title','article_fragment','carve_out_stage', 'reasoning']].merge(summary_df, left_index=True, right_index=True, how='left', suffixes=['_original', '_new'])

In [8]:
def most_common(series):
    return series.value_counts().index[0] if not series.empty else None

# Function to concatenate unique values with optional limit
def set_concat(series, max_items=None, max_chars=None):
    """
    Concatenate unique values with limits on items and characters
    """
    unique_values = pd.Series(series).dropna().astype(str).unique()
    
    # Apply item limit if specified
    if max_items and len(unique_values) > max_items:
        truncated = unique_values[:max_items]
        result = " | ".join(truncated) + f" (+ {len(unique_values) - max_items} more)"
    else:
        result = " | ".join(unique_values) if len(unique_values) > 0 else ""
    
    # Apply character limit if specified
    if max_chars and len(result) > max_chars:
        result = result[:max_chars] + "..."
        
    return result

# Group by target_company with different aggregations for each column
grouped_summary = full_df.groupby("target_company").agg({
    "group": most_common,  # Most popular group
    "group_hq": most_common,  # Most common HQ

    "vertical": most_common,  # Most common vertical

    "potential_disposal": lambda x: set_concat(x, 4),  # Concatenate unique values, limit to 4
    "potential_disposal_company": lambda x: set_concat(x, 4),  # Concatenate unique values, limit to 4
    "potential_disposal_country": lambda x: set_concat(x, 4),  # Concatenate unique values, limit to 4
    "disposal_nc_sector": most_common,  # Most common disposal sector

    "rationale": lambda x: set_concat(x, 4),  # Concatenate unique rationales, limit to 4
    "date": "min",
    "interest_score": "mean",  # Average interest score

    "carve_out_stage": most_common,  # Most common carve out stage

    "source_name": lambda x: set_concat(x, 4),  # Concatenate unique sources, limit to 4
    "title": lambda x: set_concat(x, 4),  # Concatenate unique titles, limit to 4
    "article_quote": lambda x: set_concat(x, 4),  # Concatenate unique quotes, limit to 4
    "article_fragment": lambda x: set_concat(x, max_items=4, max_chars=2500),  # Limit chars for article fragments
    
    "reasoning": lambda x: set_concat(x, 4),  # Concatenate unique reasons, limit to 4
    
})

grouped_summary['interest_score'] = grouped_summary['interest_score'].round(2)
grouped_summary['date'] = pd.to_datetime(grouped_summary['date']).dt.strftime('%Y-%m-%d')
grouped_summary.reset_index(inplace=True)
grouped_summary

,target_company,group,group_hq,vertical,potential_disposal,potential_disposal_company,potential_disposal_country,disposal_nc_sector,rationale,date,interest_score,carve_out_stage,source_name,title,article_quote,article_fragment,reasoning
0,A2A S.p.A.,A2A,IT,Energy,Gas Distribution Assets in Lombardy,Acinque SpA,IT,Other,A2A is considering divestment of non-core gas ...,2025-03-07,0.8,Early,Public Companies News and Documents via PUBT,ASCOPIAVE S.p.A. - Board of Directors Approves...,"On 30 July 2024, A2A and Ascopiave announced t...",\n\nA2A - Ascopiave joint press release\n\nOn ...,A2A (azener) has received a non-binding offer ...
1,AB Electrolux,AB Electrolux,SE,Services & Industrial Tech,Professional Division,Electrolux Professional AB,SE,Services & Industrial Tech,Electrolux is strategically refocusing by plan...,2025-05-07,0.9,Early,MarketLine Company Profiles,AB Electrolux,"In February, the company announced its plan to...","\n\nIn June, AB opened its taste center in Ban...",Electrolux has announced a future spin-off of ...
2,ABB Ltd,ABB Ltd,CH,Technology & Payments,Robotics Business,ABB Robotics,SE,Technology & Payments,ABB is considering spinning off its robotics b...,2025-04-22,0.8,Early,Cyprus Mail,ABB to spin off world’s second biggest robotic...,“We believe value creation will be higher as a...,\n\nABB said it was considering a listing in S...,ABB's CEO indicated they are considering a sta...
3,ABN AMRO,ABN AMRO,NL,Banking,Information Not Available,Information Not Available,Information Not Available,Financial Services,A strategic review led by the new CEO suggests...,2025-05-17,0.8,Early,Acquisdata Global Industry SnapShot,INDUSTRY SNAPSHOTS - NETHERLANDS BANKING 15 MA...,"In the coming period, my priority will be to l...","\n\nMarguerite Bérard, CEO:\n\n“As we reflect ...",The new CEO of ABN AMRO announced a strategic ...
4,ABN Amro,ABN Amro Group,NL,Banking,Non-core business units | Non-core units,Information Not Available,NL,Financial Services,ABN Amro is likely to divest non-core assets d...,2025-02-20,0.8,Early,Dow Jones Institutional News | Market Intellig...,Global Equities Roundup: Market Talk | KBC to ...,"""What remains to address is a wider strategic ...",\n\n1046 GMT - ABN Amro's self-help actions to...,ABN Amro is undergoing leadership change and t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
843,Zytronic PLC,Zytronic PLC,GB,Technology & Payments,Trading Entity,Zytronic Displays Ltd,GB,Technology & Payments,Zytronic PLC is considering divesting Zytronic...,2025-02-27,0.8,Late,Regulatory News Service,"Zytronic PLC Outcome Statement, Proposed De-Li...",...the Company engaged with FRP Advisory Tradi...,Zytronic PLC\n\n26 February 2025\n\n26 Februar...,Zytronic plc initiated a strategic review and ...
844,ams-OSRAM AG,ams-OSRAM AG,AT,Technology & Payments,Certain Assets | Certain assets,Information Not Available,AT | DE,Technology & Payments,ams-OSRAM is exploring strategic options for c...,2025-05-01,0.8,Early,DGAP Regulatory News,EQS-Adhoc: ams-OSRAM AG: ams OSRAM delivers 16...,The company considers strategic options for ce...,30-Apr-2025 / 07:16 CET/CEST\n\nDisclosure of ...,ams OSRAM is considering strategic options for...
845,eircom Holdings (Ireland) Ltd,eircom Group plc,IE,Technology & Payments,Information Not Available,Information Not Available,IE,Technology & Payments,The acquisition by NJJ Telecom Europe and Ilia...,2025-04-02,0.7,Early,MarketLine Company Profiles,eircom Holdings (Ireland) Ltd,NJJ Telecom Europe and Iliad announced plans t...,"\n\nIn April, the company announced an extensi...",The article mentions that NJJ Telecom Europe a...
846,u-blox Holding AG,u-blox Holding AG,CH,Technology & Payments,Cellular business,u-blox Cellular,CH,Technology & Payments,u-blox is strategically refocusing by phasing ...,2025-01-15,0.8,Early,AWP Original Press Releases | Dow Jones Instit...,EQS UBXN: u-blox Announces Strategic Decision ...,u-blox will host a Q&A session to d

#### TODO: continent from a country


In [14]:
grouped_summary = grouped_summary.loc[:,grouped_summary.columns!='reasoning']

In [9]:
from src.utils.excel_export import export_summary_to_excel
export_file = export_summary_to_excel(grouped_summary, f"../../extractions/{job_id}/summarization/carveouts_summary_grouped.xlsx")


In [17]:

from typing import Dict, List
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, Side, PatternFill
from openpyxl.utils import get_column_letter

def export_summary_to_excel(
    df: pd.DataFrame,
    excel_path: str,
    column_mapping: Dict[str, str] = {
        "target_company": "Target Company",
        "group": "Group",
        "group_hq": "Group HQ",
        "vertical": "Vertical",
        "potential_disposal": "Potential disposal (s)",
        "potential_disposal_company": "Disposal company",
        "potential_disposal_country": "Disposal country",
        "disposal_nc_sector": "Disposal NC Sector",
        "rationale": "Rationale",
        "date": "Date",
        "interest_score": "Score",
        "carve_out_stage": "Carve Out Stage",
        "article_quote": "Article quote",
        "title": "Article Title",
        "source_name": "Source Name",
        "article_fragment": "Article Fragment",
    },
    sort_by: List[str] = ["Score", "Date"],
) -> None:
    df_export = df.rename(columns=column_mapping)
    df_export = df_export.sort_values(sort_by, ascending=False)

    # Export to Excel without index column
    df_export.to_excel(excel_path, index=False)

    wb = load_workbook(excel_path)
    ws = wb.active

    # Set Arial font for entire worksheet - slightly larger font (10.5pt)
    arial_font = Font(name="Arial", size=11)
    for row in ws.rows:
        for cell in row:
            cell.font = arial_font

    # Style headers - almost black background with white text
    header_fill = PatternFill(start_color="1F1F1F", end_color="1F1F1F", fill_type="solid")
    header_font = Font(name="Arial", bold=True, color="FFFFFF", size=12)

    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal="center", vertical="center")

    # Adjust column widths based on the screenshot
    ws.column_dimensions[get_column_letter(1)].width = 28  # Target Company
    ws.column_dimensions[get_column_letter(2)].width = 18  # Group
    ws.column_dimensions[get_column_letter(3)].width = 12  # Group HQ (narrow)
    ws.column_dimensions[get_column_letter(4)].width = 16  # Vertical
    ws.column_dimensions[get_column_letter(5)].width = 30  # Potential disposal
    ws.column_dimensions[get_column_letter(6)].width = 28  # Disposal Company Name
    ws.column_dimensions[get_column_letter(7)].width = 20  # Disposal Country
    ws.column_dimensions[get_column_letter(8)].width = 28  # Disposal NC Sector
    ws.column_dimensions[get_column_letter(9)].width = 45  # Rationale (wider)
    ws.column_dimensions[get_column_letter(10)].width = 12  # Date
    ws.column_dimensions[get_column_letter(11)].width = 12  # Score (narrow)
    ws.column_dimensions[get_column_letter(12)].width = 16  # Carve Out Stage (narrow)
    ws.column_dimensions[get_column_letter(13)].width = 45  # Article Quote (wider)
    ws.column_dimensions[get_column_letter(14)].width = 20  # Article Title (wider)
    ws.column_dimensions[get_column_letter(15)].width = 30  # Article Source (wider)
    ws.column_dimensions[get_column_letter(16)].width = 65  # Article Fragment (very wide)

    # Add subtle alternating row colors - very light gray
    light_gray_fill = PatternFill(start_color="F7F7F7", end_color="F7F7F7", fill_type="solid")
    for row_idx in range(2, ws.max_row + 1):
        if row_idx % 2 == 0:  # Even rows
            for cell in ws[row_idx]:
                cell.fill = light_gray_fill

    # Set text wrapping and vertical alignment for all data cells
    for row in ws.iter_rows(min_row=2):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True, vertical="top")

    # Format date column
    for row in range(2, ws.max_row + 1):
        date_cell = ws.cell(row=row, column=10)  # Date column
        if date_cell.value:
            date_cell.number_format = "yyyy-mm-dd"
            date_cell.alignment = Alignment(horizontal="center", vertical="center")

        score_cell = ws.cell(row=row, column=11)  # Interest Score column
        if isinstance(score_cell.value, (int, float)):
            score_cell.number_format = "0.00"
            score_cell.alignment = Alignment(horizontal="center", vertical="center")

        co_stage_cell = ws.cell(row=row, column=12)
        if co_stage_cell.value:
            co_stage_cell.alignment = Alignment(horizontal="center", vertical="center")

    # Add very thin light gray borders to all cells
    thin_border = Border(
        left=Side(style="thin", color="D0D0D0"),
        right=Side(style="thin", color="D0D0D0"),
        top=Side(style="thin", color="D0D0D0"),
        bottom=Side(style="thin", color="D0D0D0"),
    )

    for row in ws.iter_rows():
        for cell in row:
            cell.border = thin_border

    # Set row height for header
    ws.row_dimensions[1].height = 24

    # Set appropriate row heights for data rows based on content
    for row in range(2, ws.max_row + 1):
        # Make rows taller to accommodate wrapped text
        ws.row_dimensions[row].height = 80  # Increased height for better quote display

    wb.save(excel_path)
    return excel_path

export_file = export_summary_to_excel(grouped_summary, f"../../extractions/{job_id}/summarization/carveouts_summary_grouped.xlsx")


In [18]:
grouped_summary.shape

(848, 16)